In [2]:
import tensorflow as tf

In [3]:
# option 1: Single element approach:  Add single element at a time:

element_list = [None]  # only ever holds 1 element at a time
dataset = tf.data.Dataset.from_generator(lambda: element_list, tf.string)

element_list[0] = "abc"

print("Output", dataset.get_single_element().numpy())

element_list[0] = "efg"

print("Output", dataset.get_single_element().numpy())


2023-03-01 10:32:48.835055: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-01 10:32:48.835133: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cccxl014): /proc/driver/nvidia/version does not exist


Output b'abc'
Output b'efg'


In [4]:
# this works for functions which change global state as well
class MyModel:
    def __init__(self):
        self.variable = tf.Variable(0)

    def change_variable(self, x):
        tf.print("Old variable", self.variable)
        self.variable.assign(x)
        tf.print("New variable", self.variable)
        return 0  # just to return a dummy value

model = MyModel()

element_list = [None]  # only ever holds 1 element at a time
dataset = tf.data.Dataset.from_generator(lambda: element_list, tf.int32)
dataset = dataset.map(model.change_variable)

element_list[0] = 100

print("Output", dataset.get_single_element().numpy())

element_list[0] = 200

print("Output", dataset.get_single_element().numpy())

Old variable 0
New variable 100
Output 0
Old variable 100
New variable 200
Output 0


In [5]:
# Option 2: Go through list of objects of arbitrary size and return numpy objects

element_list = []  # only every holds 1 element at a time
dataset = tf.data.Dataset.from_generator(lambda: element_list, tf.string)

# have to change element_list in place
element_list.clear()
element_list.extend(["abc", "efg"])

for e in dataset.as_numpy_iterator():
    print(e)

print()
# have to change element_list in place
element_list.clear()
element_list.extend(["hij", "klm", "nop"])
for e in dataset.as_numpy_iterator():
    print(e)

b'abc'
b'efg'

b'hij'
b'klm'
b'nop'


In [6]:
# this also works for functions which change global state
class MyModel:
    def __init__(self):
        self.variable = tf.Variable(0)

    def change_variable(self, x):
        tf.print("Old variable", self.variable)
        self.variable.assign(x)
        tf.print("New variable", self.variable)
        return 0  # just to return a dummy value

model = MyModel()

element_list = [None]  # only ever holds 1 element at a time
dataset = tf.data.Dataset.from_generator(lambda: element_list, tf.int32)
dataset = dataset.map(model.change_variable)

# have to change element_list in place
element_list.clear()
element_list.extend([100, 200])

for e in dataset.as_numpy_iterator():
    print(e)

print()
# have to change element_list in place
element_list.clear()
element_list.extend([300, 400, 500])
for e in dataset.as_numpy_iterator():
    print(e)

Old variable 0
New variable 100
0
Old variable 100
New variable 200
0

Old variable 200
New variable 300
0
Old variable 300
New variable 400
0
Old variable 400
New variable 500
0
